In [14]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [3]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

In [5]:
# Setup for train dataset

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(128)
ds_test = ds_train.prefetch(AUTOTUNE)

In [54]:
model = keras.Sequential(
    [
        keras.Input((28,28,1)),
        layers.Conv2D(32,3,activation="relu"),
        layers.Flatten(),
        layers.Dense(10, activation="softmax")
    ]
)

num_epochs = 1
optimizer = keras.optimizers.Adam()
# loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
acc_metric = keras.metrics.SparseCategoricalAccuracy()


In [55]:
# Training Loop
for epoch in range(num_epochs):
    print(f"\nStart of Training Epoch {epoch}")
    for batch_idx, (x_batch, y_true) in enumerate(ds_train):
        with tf.GradientTape() as tape:

            # my_stuff = 
            y_pred = model(x_batch, training=True)
            my_stuff = np.random.rand(128,10)
            my_stuff = tf.convert_to_tensor(my_stuff)
            my_stuff = tf.cast(my_stuff, dtype=tf.float32)
            # print(my_stuff)
            # print(y_pred)
            print("y_true orig: ")
            print(y_true)
            print("y_pred orig: ")
            print(y_pred)

            y_true = np.random.randint(1, 10, size=128)
            y_true = tf.constant(y_true, shape=(128,))
            y_true = tf.cast(y_true, dtype=tf.int64)

            y_pred = np.random.sample((128,10))
            # y_pred = tf.constant(y_pred, shape=(128,))
            y_pred = tf.cast(y_pred, dtype=tf.float32)
            print("y_pred: ")
            print(y_pred)




            # y_pred = tf.math.multiply(my_stuff, y_pred)
            loss = loss_fn(y_true, y_pred)
            print("y_true: ")
            print(y_true)


        gradients = tape.gradient(loss, model.trainable_weights)
        print(gradients)

        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        acc_metric.update_state(y_true, y_pred)
    train_acc = acc_metric.result()
    print(f"Accuracy over epoch {train_acc}")
    acc_metric.reset_states()



Start of Training Epoch 0
y_true orig: 
tf.Tensor(
[4 4 7 1 1 6 7 8 3 5 9 5 2 5 6 0 1 7 6 3 2 0 9 3 3 1 4 8 4 5 0 1 7 3 0 1 1
 8 2 2 8 5 1 8 3 9 9 9 6 1 1 7 6 5 4 0 6 7 8 3 3 6 0 2 5 5 1 7 0 1 2 6 2 5
 3 9 1 1 0 1 1 1 2 8 7 3 5 2 1 1 5 5 7 2 7 5 9 6 7 6 5 2 3 6 1 9 2 6 7 1 2
 4 2 2 0 5 8 3 6 6 3 8 1 1 7 1 8 0], shape=(128,), dtype=int64)
y_pred orig: 
tf.Tensor(
[[0.09683486 0.08825424 0.10060406 ... 0.10174596 0.09697383 0.10234646]
 [0.10176815 0.09848714 0.09924846 ... 0.10073833 0.09847818 0.09899288]
 [0.0995769  0.10280705 0.10460467 ... 0.09668235 0.10352203 0.09575957]
 ...
 [0.09595897 0.10591817 0.10291444 ... 0.10015039 0.09708726 0.10229485]
 [0.10203396 0.10016441 0.10336605 ... 0.09956773 0.09845336 0.10151705]
 [0.10177679 0.10289359 0.1028979  ... 0.10762431 0.09446923 0.09803254]], shape=(128, 10), dtype=float32)
y_pred: 
tf.Tensor(
[[0.3925746  0.02695567 0.76566666 ... 0.04176499 0.6756355  0.26980376]
 [0.3820538  0.24528734 0.21503788 ... 0.946764   0.86451083 0.0

ValueError: logits and labels must have the same shape ((128, 10) vs (128,))

In [ ]:
# Test Loop
for batch_idx, (x_batch, y_batch) in enumerate(ds_test):
    y_pred = model(x_batch, training=False)
    acc_metric.update_state(y_batch, y_pred)

train_acc = acc_metric.result()
print(f"Accuracy over Test Set: {train_acc}")
acc_metric.reset_states()

Accuracy over TEst Set: 0.9921500086784363
